In [1]:
# RF_5_pg_acc = [0.9575, 0.94, 0.9575, 0.95, 0.9, 0.9325, 0.93, 0.96, 0.9175, 0.9325, 0.895, 0.94]
# DT_5_pg_acc = [0.9675, 0.97, 0.9675, 0.98, 0.8725, 0.9625, 0.955, 0.9775, 0.95, 0.95, 0.9525, 0.955]
# DT_5_mssql_acc = [0.985, 0.9775, 0.875, 0.835, 0.935, 0.9825, 0.98, 0.965, 0.9925, 0.955, 0.985, 0.99]
# RF_5_mssql_acc = [0.985, 0.98, 0.8675, 0.8325, 0.91, 0.98, 0.98, 0.9625, 0.9875, 0.9625, 0.9925, 0.985]
# MLP_mssql_acc = [0.985, 0.9525, 0.865, 0.8425, 0.975, 0.9875, 0.965, 0.95, 0.9675, 0.9875, 0.9925, 0.985]

In [1]:
from core.DataLoader import *
from core.models.MLP import * 
from core.models.GAM import *
from core.models.SVM import *
# from core.models.EBM import *

from core.models.RandomForest import *
from core.Visualizer import *
from core.models.Classifier import *
from core.models.XGB import *

In [2]:
model_accs = {
    'MLP': [],
    'RF(5)': [],
    'RF(10)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(10)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}
# mssql_accs = {
#     'MLP': [],
#     'RF(5)': [],
#     'RF(10)': [],
#     'RF(U)': [],
#     'DT(5)': [],
#     'DT(10)': [],
#     'DT(U)': [],
#     'LR': [],
#     'SVM': [],
#     'XGB': []
# }
# present_idxs = [0, 1, 2, 3, 4, 5, 8 ,12, 21, 22, 23, 24]

In [3]:
weighted_clf_model_list ={
    "MLP": MLPClassifier,
    "DT": m_DecisionTreeClassifier,
    "RF": m_RandomForestClassifier,
    "LR": m_LogisticRegression,
    "SVM": SVMClassifier,
    "XGB": XGBOOSTClassifier
}
# weighted_clf_model_name_list = ['MLP', 'RF(5)', 'RF(U)', 'DT(5)', 'DT(U)', 'LR', 'SVM', 'XGB']
# weighted_clf_model_name_list = ['MLP', 'RF(U)', 'DT(U)']
# weighted_clf_model_name_list = ['DT(5)', 'RF(5)',  'MLP']
# weighted_clf_model_name_list = ['DT(5)', 'RF(5)']
weighted_clf_model_name_list = ['MLP']

In [5]:
engine = 'mssql'
# if engine == 'mssql':
#     baseline = mssql_accs
# else:
#     baseline = postgres_accs
dl = DataLoader(engine)
one_file_dss, one_file_names = dl.get_one_file_ds(return_type='ds and names', 
                                                  datasets=['ssb', 'tpch', 'tpch_10', 'tpch_100', 'imdb'],
                                                 suffix='(random_left)')
# present_idxs = range(len(one_file_dss))

In [6]:
all_features = dl.all_features
classification_target = dl.classification_target
all_features  = dl.base_features
# sel_of_join_pred
# key_feature_set = 0
# features = key_features[key_feature_set]
# print("Using features: ", features)

In [7]:
removed_acc = {}
reset_accs = {
    'MLP': [],
    'RF(5)': [],
    'RF(U)': [],
    'DT(5)': [],
    'DT(U)': [],
    'LR': [],
    'SVM': [],
    'XGB': []
}

feature_groups = [
    ['left_cardinality', 'base_cardinality', 'result_size'],
    ['sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'sel_of_pred_on_indexed_attr_and_join_pred',
     'sel_of_pred_on_non_indexed_attr_and_join_pred', 'sel_of_pred_on_indexed_attr_and_non_indexed_attr',
     'total_sel_on_base_table', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr'],
    [ 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed']    
]

In [8]:
present_idxs = list(range(len(one_file_dss)))
# present_idxs = [0]

In [9]:
all_features = ['left_cardinality',
 'base_cardinality',
 'sel_of_pred_on_indexed_attr',
 'sel_of_pred_on_non_indexed_attr',
 'left_ordered',
 'base_ordered',
 'left_indexed',
 'base_indexed',
 'result_size',
 'predicate_op_num_on_indexed_attr',
 'predicate_op_num_on_non_indexed_attr']

# Base accuracy with all features

In [10]:
# # present_idxs = list(range(len(one_file_dss)))    
# features = all_features.copy()
    

# full_model_accs = {
#     'MLP': [],
#     'RF(5)': [],
#     'RF(U)': [],
#     'DT(5)': [],
#     'DT(U)': [],
#     'LR': [],
#     'SVM': [],
#     'XGB': []
# }

# for i in present_idxs:  # range(len(one_file_dss)):
# #     i = 0
#     print(f"Processing: {i}")
#     # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
#     ds = one_file_dss[i]
#     ds_name = one_file_names[i]

#     # =========================
#     X = ds[features]
#     y = ds['optimal_decision']
#     X_costs = ds[dl.regression_targets]

#     from sklearn.model_selection import train_test_split
#     from sklearn import preprocessing

#     X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
#     X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
#     X_train, X_test, y_train, y_test = \
#         X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

#     X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())

#     scaler = preprocessing.StandardScaler().fit(X_train)
#     X_train = scaler.transform(X_train)

#     # scaler = preprocessing.StandardScaler().fit(X_test)
#     X_test = scaler.transform(X_test)

#     X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
#     # =========================

#     accs = []
#     for idx, model_name in enumerate(weighted_clf_model_name_list):
#         model = weighted_clf_model_list[model_name.split('(')[0]]        
#         if 'DT' in model_name or 'RF' in model_name:
#             max_depth = model_name.split('(')[-1].split(')')[0]
#             if max_depth.lower() == 'u':
#                 max_depth = None
#             else:
#                 max_depth = int(max_depth)
#             clf = model().fit(X_train, y_train, sample_weight=None, max_depth=max_depth)
#         else:
#             clf = model().fit(X_train, y_train, sample_weight=None, max_iter=200, weight_decay=0.00001)

#         print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
# #         if len(features) == 2:
# #             plot_2d_decision_boundaries(clf, scaler.inverse_transform(X_train), X_train_costs, y_train, title=f'{model} on {one_file_names[i]}')

        
#         full_model_accs[model_name].append(clf.score(X_test, y_test))
# #         accs.append(clf.score(X_test, y_test))

  0%|          | 0/200 [00:00<?, ?it/s]

Processing: 0
set features: 11


  0%|          | 0/200 [00:00<?, ?it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9825
Processing: 1
set features: 11


  0%|          | 0/200 [00:00<?, ?it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.995
Processing: 61
set features: 11


  0%|          | 0/200 [00:00<?, ?it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.98
Processing: 62
set features: 11


100%|██████████| 200/200 [00:36<00:00,  5.53it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9625


# Remove feature acc

In [10]:
removed_acc = {}

for feat_idx, feat_in_group in enumerate(all_features):
    
    features = all_features.copy()
    
    features.remove(feat_in_group)
    
    print(f"remove feature set{feat_idx}: after remove: {features}")
    
    model_accs = {
        'MLP': [],
        'RF(5)': [],
        'RF(U)': [],
        'DT(5)': [],
        'DT(U)': [],
        'LR': [],
        'SVM': [],
        'XGB': []
    }
    
    for i in present_idxs:  # range(len(one_file_dss)):
    #     i = 0
        print(f"Feature {feat_idx} / {len(all_features)}; Processing: {i} / {len(present_idxs)}")

        # ds = one_file_dss[i][one_file_dss[i]['predicate_op_num_on_non_indexed_attr'] == 0]
        ds = one_file_dss[i]
        ds_name = one_file_names[i]

        # =========================
        X = ds[features]
        y = ds['optimal_decision']
        X_costs = ds[dl.regression_targets]

        from sklearn.model_selection import train_test_split
        from sklearn import preprocessing

        X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=1)
        X_train_costs, _, _, _, = train_test_split(X_costs, y, train_size=0.8, random_state=1)
        X_train, X_test, y_train, y_test = \
            X_train.to_numpy(),X_test.to_numpy(), y_train.to_numpy(), y_test.to_numpy()

        X_train_weights = calculate_importance_from_costs(X_train_costs.to_numpy())

        scaler = preprocessing.StandardScaler().fit(X_train)
        X_train = scaler.transform(X_train)

        # scaler = preprocessing.StandardScaler().fit(X_test)
        X_test = scaler.transform(X_test)

        X_train_weights = preprocessing.MinMaxScaler().fit_transform(X_train_weights.reshape(-1,1)).flatten()
        # =========================

        accs = []
        for idx, model_name in enumerate(weighted_clf_model_name_list):
            model = weighted_clf_model_list[model_name.split('(')[0]]        
            if 'DT' in model_name or 'RF' in model_name:
                max_depth = model_name.split('(')[-1].split(')')[0]
                if max_depth.lower() == 'u':
                    max_depth = None
                else:
                    max_depth = int(max_depth)
                clf = model().fit(X_train, y_train, sample_weight=None, max_depth=max_depth)
            else:
                clf = model().fit(X_train, y_train, sample_weight=None, max_iter=100, weight_decay=0.00001)

            print(f"Accuray of {model}: {clf.score(X_test, y_test)}")
    #         if len(features) == 2:
    #             plot_2d_decision_boundaries(clf, scaler.inverse_transform(X_train), X_train_costs, y_train, title=f'{model} on {one_file_names[i]}')

            model_accs[model_name].append(clf.score(X_test, y_test))
    #         accs.append(clf.score(X_test, y_test))    
    removed_acc["%d" % feat_idx] = model_accs

  0%|          | 0/100 [00:00<?, ?it/s]

remove feature set0: after remove: ['base_cardinality', 'sel_of_pred_on_indexed_attr', 'sel_of_pred_on_non_indexed_attr', 'left_ordered', 'base_ordered', 'left_indexed', 'base_indexed', 'result_size', 'predicate_op_num_on_indexed_attr', 'predicate_op_num_on_non_indexed_attr']
Feature 0 / 11; Processing: 0 / 63
set features: 10


  0%|          | 0/100 [00:00<?, ?it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9425
Feature 0 / 11; Processing: 1 / 63
set features: 10


  0%|          | 0/100 [00:00<?, ?it/s]

...

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.9575
Feature 10 / 11; Processing: 61 / 63
set features: 10


  0%|          | 0/100 [00:00<?, ?it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.965
Feature 10 / 11; Processing: 62 / 63
set features: 10


100%|██████████| 100/100 [00:09<00:00, 10.70it/s]

Accuray of <class 'core.models.MLP.MLPClassifier'>: 0.97


In [12]:
# removed_acc.keys()

In [13]:
# import json
# with open('acc_drop_mssql.json', 'w') as fp:
#     json.dump(removed_acc, fp)

In [14]:
# import json 
  
# # Opening JSON file 
# with open('removed_acc_postgres.json', 'r') as json_file: 
#     removed_acc = json.load(json_file) 

In [15]:
# removed_acc_MLP
all_features

['left_cardinality',
 'base_cardinality',
 'sel_of_pred_on_indexed_attr',
 'sel_of_pred_on_non_indexed_attr',
 'left_ordered',
 'base_ordered',
 'left_indexed',
 'base_indexed',
 'result_size',
 'predicate_op_num_on_indexed_attr',
 'predicate_op_num_on_non_indexed_attr']

# Feature rank per binary join

In [11]:
top_feature_chosen = {}
for model_name in weighted_clf_model_name_list:
    model_features = []
    for i in range(len(present_idxs)):
#         full_acc = full_model_accs[model_name][i]
        ablation_accs = []
        for feat_idx in range(len(all_features)):
            ablation_accs.append(removed_acc["%d" % feat_idx][model_name][i])
        ablation_accs = np.array(ablation_accs)
        if sum(ablation_accs) == ablation_accs[0] * len(ablation_accs):
            ablation_accs[range(len(ablation_accs))] = 0
        else:  
            sort_idx = np.argsort(ablation_accs)
            ablation_accs[sort_idx[0:2]] = 1
            ablation_accs[sort_idx[2:]] = 0
        model_features.append(ablation_accs)
#         print(ablation_accs)
    top_feature_chosen[model_name] = model_features

In [12]:
for model_name in weighted_clf_model_name_list:
    print(f"For model {model_name}")
    sum_feat = np.zeros(len(all_features))
    for idx, ds_name in zip(range(len(one_file_dss)), one_file_names):
        print(f"{ds_name}: ", top_feature_chosen[model_name][idx])
        sum_feat += top_feature_chosen[model_name][idx]
    print(f"Summary for {model_name}: ", sum_feat)
    print()

For model MLP
ssb part/mssql_lineorder_part_optimal.csv:  [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0.]
ssb customer/mssql_lineorder_customer_optimal.csv:  [0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0.]
ssb ddate/mssql_lineorder_ddate_optimal.csv:  [0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0.]
ssb supplier/mssql_lineorder_supplier_optimal.csv:  [1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
ssb lineorder/mssql_part_lineorder_optimal.csv:  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ssb lineorder/mssql_customer_lineorder_optimal.csv:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
ssb lineorder/mssql_ddate_lineorder_optimal.csv:  [0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
ssb lineorder/mssql_supplier_lineorder_optimal.csv:  [1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
tpch customer/mssql_orders_customer_optimal.csv:  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
tpch orders/mssql_customer_orders_optimal.csv:  [1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
tpch orders/mssql_lineitem_orders_optimal.csv:  [0. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
tpch part/mssql_lineitem_part_optimal.csv:  [0. 0. 0. 0.

# Average the model performance drop

## DT drop

In [18]:
# dt_drop = []

# for idx, feat_t in enumerate(all_features):
#     acc = removed_acc["%d" % idx]['DT(5)']
#     drops = []
#     for i in range(len(present_idxs)):
#         drops.append(acc[i] - DT_5_mssql_acc[i])
#     dt_drop.append(-np.average(drops))
#     print(f"Avg drop for {feat_t}: {np.average(drops)}")
    

In [19]:
# rf_drop = []

# for idx, feat_t in enumerate(all_features):
#     acc = removed_acc["%d" % idx]['RF(5)']
#     drops = []
#     for i in range(len(present_idxs)):
#         drops.append(acc[i] - RF_5_mssql_acc[i])
#     rf_drop.append(-np.average(drops))
#     print(f"Avg drop for {feat_t}: {np.average(drops)}")

In [20]:
# mlp_drop = []

# for idx, feat_t in enumerate(all_features):
#     acc = removed_acc["%d" % idx]['MLP']
#     drops = []
#     for i in range(len(present_idxs)):
#         drops.append(acc[i] - MLP_mssql_acc[i])
#     mlp_drop.append(-np.average(drops))
#     print(f"Avg drop for {feat_t}: {np.average(drops)}")

In [21]:
# mlp_drop = np.array(mlp_drop)
# dt_drop = np.array(dt_drop)
# rf_drop = np.array(rf_drop)

# # mssql_imp = overall_imp[1]
# # shap_values = shap_values / len(present_idxs)
# # importances = np.array([feature_scores[f] for f in features])

# # ===========================================
# # indices = np.argsort(importances)
# # ===========================================
# barWidth = 0.5
# # indices = [11, 10, 1, 12,  9, 8, 7, 6, 5, 4, 3, 2, 0]
# indices = list(range(len(all_features)))
# # indices = [2,1,0]

# indices.reverse()


# import matplotlib
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42

# plt.figure(figsize=(18, 10), dpi=300)

# # plt.title('Feature Importances')
# r1 = [i * 2 for i in range(len(indices))]
# r2 = [x + barWidth for x in r1]
# r3 = [x + barWidth for x in r2]

# plt.barh(r1, rf_drop[indices], height=barWidth, color='tab:blue', align='center',  label='RF-5')
# # plt.barh(r1 , mssql_imp[indices], height=barWidth , color='tab:orange', align='center', label='Microsoft SQL Server')
# plt.barh(r2, dt_drop[indices], height=barWidth, color='tab:orange', align='center',  label='DT-5')
# plt.barh(r3, mlp_drop[indices], height=barWidth, color='tab:purple', align='center',  label='MLP')

# # group_names = ['Group 1: cardinalities', 'Group 2: selectivities', 'Group 3: indices and orders']

# plt.yticks([i + 1 * barWidth for i in r1], [all_features[i] for i in indices])
# plt.xlabel('Accuracy Drop', fontsize=30)

# axes = plt.gca()
# axes.tick_params(axis='x', which='major', labelsize=30)
# axes.tick_params(axis='y', which='major', labelsize=30)

# plt.legend(loc='lower right', prop={'size': 30})

# plt.grid(axis='x')
# plt.tight_layout()
# plt.savefig(f'./figures/exp2-mssql-ablation-features.pdf')
# plt.show()

## RF

In [22]:
# dt_drop = []

# for idx, feat_t in enumerate(all_features):
#     acc = removed_acc["%d" % idx]['RF(5)']
#     drops = []
#     for i in range(len(present_idxs)):
#         drops.append(acc[i] - postgres_accs[1][i])
#     dt_drop.append(-np.average(drops))
#     print(f"Avg drop for {feat_t}: {np.average(drops)}")

In [23]:
# postgre_imp = np.array(dt_drop)
# # mssql_imp = overall_imp[1]
# # shap_values = shap_values / len(present_idxs)
# # importances = np.array([feature_scores[f] for f in features])

# # ===========================================
# # indices = np.argsort(importances)
# # ===========================================
# barWidth = 0.4
# indices = [11, 10, 1, 12,  9, 8, 7, 6, 5, 4, 3, 2, 0]

# import matplotlib
# matplotlib.rcParams['pdf.fonttype'] = 42
# matplotlib.rcParams['ps.fonttype'] = 42

# plt.figure(figsize=(18, 8), dpi=300)

# # plt.title('Feature Importances')
# r1 = [i for i in range(13)]
# # r2 = [x + barWidth for x in r1]

# plt.barh(r1, postgre_imp[indices], height=barWidth, color='tab:blue', align='center',  label='PostgreSQL RF-5')
# # plt.barh(r1 , mssql_imp[indices], height=barWidth , color='tab:orange', align='center', label='Microsoft SQL Server')



# plt.yticks(range(len(indices)), [dl.all_features[i] for i in indices])
# plt.xlabel('Accuracy Drop', fontsize=25)

# axes = plt.gca()
# axes.tick_params(axis='x', which='major', labelsize=25)
# axes.tick_params(axis='y', which='major', labelsize=25)

# plt.legend(loc='lower right', prop={'size': 25})

# plt.grid(axis='x')
# plt.tight_layout()
# plt.savefig(f'./figures/exp2-RF-ablation.pdf')
# plt.show()